In [1]:

from ipywidgets import widgets



#Menu Coefficients
Menu_Coefficients = widgets.SelectMultiple (
    description = "Coefficients",
    options=['Coefficients entiers', 'Coefficients rationnels'],
    value=['Coefficients rationnels'],
    #rows=10,
    disabled=False
    )    
#Menu Fonctions
Menu_Fonctions = widgets.SelectMultiple (
    description = "Fonctions utilisées",
    options=['puissances positives', 'puissances négatives', 'racine carrée', 'cosinus et sinus','exponentielle', 'logarithme népérien'],
    value=['puissances positives', 'puissances négatives', 'racine carrée'],
    #rows=10,
    disabled=False
    ) 
#Menu Formules utilisées
Menu_Formules = widgets.SelectMultiple (
    description = "Formules utilisées",
    options=['Produit par un scalaire', 'Somme de fonctions', 'Produit de fonctions', 'Quotient de fonctions','Composition par un polynôme'],
    value=['Produit par un scalaire', 'Somme de fonctions', 'Produit de fonctions', 'Quotient de fonctions'],
    #rows=10,
    disabled=False
    ) 

Menu_accordeon = widgets.Accordion(children = [Menu_Coefficients, Menu_Fonctions, Menu_Formules])

Menu_accordeon.titles = (children.description for children in Menu_accordeon.children)

#titles = ("Coefficients","Fonctions utilisées","Formules utilisées")                          
                         
                          
Menu_accordeon       

Accordion(children=(SelectMultiple(description='Coefficients', index=(1,), options=('Coefficients entiers', 'C…